# TO USE GPU

USE GOOGLE COLLAB AND MOUNT YOUR DRIVE BY UNCOMMENTING THE FOLLOWING CODE (Ensure image folder is in your google drive)

Also, to enable the GPU on google colab, go to runtime tab and change runtime and add accelerator GPU

In [16]:
## Mount google drive
#from google.colab import drive
#drive.mount("/content/gdrive")

# Convolution Neural Network - CNN Baseline

Convolution Neural Networks or CNN or Convnets are the current state of the art for most computer vision tasks.

This notebook will aply k-fold and use the concept of transfer learning.

Transfer learning:
- Train a CNN using 6 emotions dataset known as FER2013 dataset
- Strip out the top layers of that trained CNN architecture
- Freeze the weights of that architecture in hopes it learned high level features already
- Add back other layers that are trainable
- Train using our Smile dataset to learn specific features

In [17]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Plot architecture
import keras
import pydot
from keras.utils.vis_utils import model_to_dot

from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os
from PIL import Image

# Data  
  
The first step is to import images and store their **pixel** values in a dataframe. This will allow us to create 10 fold by using the index of the rows.  
  
We will also **shuffle** the dataframe because the first 200 images are smilling and next 200 images are neutral.

In [18]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('./SMILE_Dataset/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('./SMILE_Dataset/all/'+fname, target_size=(64, 64))) for fname in labels['fname']])

# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded

Now we have two variables.  
  
x: all of the values for our images  
y: all of the labels (0:1)
  
Now we have finished working with the data. Let's define an architecture for our CNN.

# Models

In [19]:
def pizza_model():
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(70, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

# K-Fold

In [20]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []

# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))

    id += 1 
    
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    model = pizza_model()
    
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.1)   
    
    datagen.fit(X_train)

    # Secret sauce to get 3-5 % accuracy more
    # Adjust the learning rate over time. (Like we saw in class!)
    # The learning rate determines the size of the steps taken during the gradient descent process.
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Used to prevent overfitting. 
    # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 20), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 20,
                              callbacks=[learning_rate_reduction])

    
    y_pred = model.predict(X_test)
    y_pred = [np.round(p[0]) for p in y_pred]
    
    print(classification_report(y_test, y_pred))
    reports.append(classification_report(y_test, y_pred,output_dict=True))
    
#     if id == 3:
#         break

Kfold iteration 1/10
Total images: 400 ---- Train images: 360 ---- Test images: 40
Epoch 1/20
18/18 [==============================] - 2s 133ms/step - loss: 0.7016 - acc: 0.5083 - val_loss: 4.6803 - val_acc: 0.5000
Epoch 2/20
18/18 [==============================] - 2s 103ms/step - loss: 0.6859 - acc: 0.5917 - val_loss: 3.0707 - val_acc: 0.6000
Epoch 3/20
18/18 [==============================] - 2s 102ms/step - loss: 0.6139 - acc: 0.7000 - val_loss: 26.4008 - val_acc: 0.8250
Epoch 4/20
18/18 [==============================] - 2s 98ms/step - loss: 0.3940 - acc: 0.8389 - val_loss: 28.6128 - val_acc: 0.9000
Epoch 5/20
18/18 [==============================] - 2s 99ms/step - loss: 0.2653 - acc: 0.8750 - val_loss: 18.8471 - val_acc: 0.9500
Epoch 6/20
18/18 [==============================] - 2s 99ms/step - loss: 0.2052 - acc: 0.9139 - val_loss: 12.5188 - val_acc: 0.9500
Epoch 7/20
18/18 [==============================] - 2s 100ms/step - loss: 0.1548 - acc: 0.9417 - val_loss: 31.2820 - val_acc

18/18 [==============================] - 2s 97ms/step - loss: 0.1758 - acc: 0.9389 - val_loss: 51.9203 - val_acc: 0.8500
Epoch 9/20
18/18 [==============================] - 2s 100ms/step - loss: 0.1531 - acc: 0.9556 - val_loss: 50.9158 - val_acc: 0.8750
Epoch 10/20
18/18 [==============================] - 2s 100ms/step - loss: 0.1549 - acc: 0.9333 - val_loss: 59.6245 - val_acc: 0.8500

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 11/20
18/18 [==============================] - 2s 100ms/step - loss: 0.1522 - acc: 0.9389 - val_loss: 51.5934 - val_acc: 0.8500
Epoch 12/20
18/18 [==============================] - 2s 102ms/step - loss: 0.1516 - acc: 0.9333 - val_loss: 50.0852 - val_acc: 0.8500
Epoch 13/20
18/18 [==============================] - 2s 100ms/step - loss: 0.1437 - acc: 0.9389 - val_loss: 48.3903 - val_acc: 0.8750

Epoch 00013: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 14/20
18/18 [==============================

18/18 [==============================] - 2s 100ms/step - loss: 0.1183 - acc: 0.9500 - val_loss: 3.1125 - val_acc: 0.9750
Epoch 17/20
18/18 [==============================] - 2s 102ms/step - loss: 0.1306 - acc: 0.9444 - val_loss: 4.4940 - val_acc: 0.9750
Epoch 18/20
18/18 [==============================] - 2s 102ms/step - loss: 0.1050 - acc: 0.9694 - val_loss: 9.9147 - val_acc: 0.9750

Epoch 00018: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 19/20
18/18 [==============================] - 2s 102ms/step - loss: 0.1225 - acc: 0.9500 - val_loss: 6.9578 - val_acc: 0.9750
Epoch 20/20
18/18 [==============================] - 2s 100ms/step - loss: 0.1222 - acc: 0.9583 - val_loss: 7.0247 - val_acc: 0.9750
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      0.95      0.97        20

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97      

Epoch 3/20
18/18 [==============================] - 2s 112ms/step - loss: 0.4475 - acc: 0.7944 - val_loss: 20.6232 - val_acc: 0.9000
Epoch 4/20
18/18 [==============================] - 2s 113ms/step - loss: 0.2567 - acc: 0.9083 - val_loss: 12.3599 - val_acc: 0.9500
Epoch 5/20
18/18 [==============================] - 2s 106ms/step - loss: 0.1724 - acc: 0.9278 - val_loss: 13.3042 - val_acc: 0.9250
Epoch 6/20
18/18 [==============================] - 2s 107ms/step - loss: 0.1916 - acc: 0.9472 - val_loss: 10.3632 - val_acc: 0.9500
Epoch 7/20
18/18 [==============================] - 2s 112ms/step - loss: 0.1401 - acc: 0.9361 - val_loss: 6.6844 - val_acc: 0.9500

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 8/20
18/18 [==============================] - 2s 95ms/step - loss: 0.1526 - acc: 0.9444 - val_loss: 4.2663 - val_acc: 0.9750
Epoch 9/20
18/18 [==============================] - 2s 104ms/step - loss: 0.1362 - acc: 0.9444 - val_loss: 9.3615 - val_acc:

18/18 [==============================] - ETA: 0s - loss: 0.1434 - acc: 0.941 - 2s 103ms/step - loss: 0.1479 - acc: 0.9417 - val_loss: 24.1191 - val_acc: 0.9250

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 13/20
18/18 [==============================] - 2s 93ms/step - loss: 0.1279 - acc: 0.9528 - val_loss: 22.3167 - val_acc: 0.9500
Epoch 14/20
18/18 [==============================] - 2s 110ms/step - loss: 0.1546 - acc: 0.9333 - val_loss: 20.2055 - val_acc: 0.9500
Epoch 15/20
18/18 [==============================] - 2s 110ms/step - loss: 0.1214 - acc: 0.9556 - val_loss: 18.8691 - val_acc: 0.9500

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 16/20
18/18 [==============================] - 2s 110ms/step - loss: 0.1311 - acc: 0.9444 - val_loss: 26.8081 - val_acc: 0.9250
Epoch 17/20
18/18 [==============================] - 2s 110ms/step - loss: 0.1361 - acc: 0.9444 - val_loss: 24.3113 - val_acc: 0.9250
Epoch 18/

In [21]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))

Final F1-Score is: 95.5%
